In [ ]:
import glob
import os

import h5py
import holoviews as hv
import numpy as np
import pandas as pd
import panel as pn
import param
import yaml
from holoviews import opts
from scipy.constants import c, physical_constants
from tqdm import tqdm

hv.extension("bokeh", "matplotlib")
from bokeh.io import export_png, export_svgs

opts.defaults(
    opts.Scatter(width=1000, height=300, tools=["hover"]),
    opts.Histogram(width=1000, height=300, tools=["hover"]),
    opts.Image(width=1000, height=300, tools=["hover"]),
    opts.Curve(width=1000, height=300, tools=["hover"]),
    opts.Points(width=1000, height=300, tools=["hover"]),
)


%pylab inline
# from matplotlib.colors import LogNorm
%config InlineBackend.figure_format ='retina'

rcParams["figure.figsize"] = (13.0, 6.0)

from scipy.optimize import curve_fit
from scipy.stats import norm


def get_data_pd(fname: str) -> pd.DataFrame:
    try:
        with h5py.File(fname, "r") as f:
            rawNr = f["raw/trigger nr"][:]
            rawTof = f["raw/tof"][:] * 1e6
            rawTot = f["raw/tot"][:]
            rawX = f["raw/x"][:]
            rawY = f["raw/y"][:]
            centNr = f["centroided/trigger nr"][:]
            centTof = f["centroided/tof"][:] * 1e6
            centTot = f["centroided/tot max"][:]
            centY = f["centroided/y"][:]
            centX = f["centroided/x"][:]
            size = f["centroided/clustersize"][:]

        raw_data = pd.DataFrame(
            np.column_stack((rawNr, rawTof, rawTot, rawX, rawY)),
            columns=("nr", "tof", "tot", "x", "y"),
        )
        cent_data = pd.DataFrame(
            np.column_stack((centNr, centTof, centTot, centX, centY, size)),
            columns=("nr", "tof", "tot", "x", "y", "size"),
        )
        return raw_data, cent_data
    except:
        print(f'key "{keys}" not known or file "{fname}" not existing')


def gauss_fwhm(x, *p):
    A, mu, fwhm = p
    return A * np.exp(-((x - mu) ** 2) / (2.0 * (fwhm ** 2) / (4 * 2 * np.log(2))))


def find_peaks_in_microbunch(
    data: pd.DataFrame, nr_peaks: int = 4, dt: float = 10, offset: float = 0
) -> list:
    """find first peak in micro-bunch"""
    peaks = []
    for i in range(nr_peaks):
        mask = np.logical_and(
            data["tof"] > (offset + i * dt), data["tof"] < (offset + i * dt + 1)
        )
        x_hist, x_edges = np.histogram(data["tof"][mask], bins=1_000)
        x = (x_edges[:-1] + x_edges[1:]) * 0.5
        popt, pcov = curve_fit(
            gauss_fwhm, x, x_hist, p0=[x_hist.max(), x[x_hist.argmax()], 0.05]
        )
        peaks.append(popt[1])
    return peaks


def shift_microbunch_pulses(
    data: pd.DataFrame, nr_peaks: int = 4, dt: float = 10, offset: float = 0
) -> pd.DataFrame:
    """Fold consecutive micro-bunch pulses back to first"""
    peaks = find_peaks_in_microbunch(data, nr_peaks, dt, offset)

    # shift bunches
    for i in range(1, nr_peaks):
        mask = np.logical_and(
            data["tof"] >= offset + i * dt, data["tof"] < offset + (i + 1) * dt
        )
        data["tof"][mask] -= peaks[i] - peaks[0]

    return data


def radial_profile(data: np.array, center: tuple) -> np.array:
    y, x = np.indices(data.shape)
    r = np.sqrt((x - center[0]) ** 2 + (y - center[1]) ** 2)
    r = r.astype(np.int)

    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    return radialprofile


get_x_axis_from_bins = lambda x_bins: 0.5 * (x_bins[1:] + x_bins[:-1])
file_title = lambda x: os.path.basename(x).rstrip(".hdf5")

with open("runs.yaml") as f:
    runNrs = yaml.safe_load(f)

In [ ]:
opts.defaults(
    opts.Image(frame_width=400, frame_height=400, colorbar=True, cmap="jet", logz=False)
)

#### define constants and conversion functions

In [ ]:
#######
# physical constants
u = physical_constants["atomic mass constant"][0]
J2eV = 1 / physical_constants["electron volt-joule relationship"][0]
e = physical_constants["elementary charge"][0]  # C
u = physical_constants["atomic mass constant"][0]
E = 269.5 * 100  # 158.9 * 100  # V/m, from Benjamins Simion simulation
m = 14.0067 * u  # kg

##########
# constants from experiment
x_cent, y_cent, detector_radius = 136, 140, 112
tof_center = 2.4213223243819346  # extracted from same variable in CH3I_TOFs.ipynb
# https://www.wolframalpha.com/input/?i=26.5713+mm%2F%2810+mm%2Fus+*+2.2567+us%29 from Benjamin Email
vmi_magnification = 28.6445 / (10 * 2.4275)
c_pixel = (
    78 / (2 * detector_radius) * 1e-3
)  # 78mm / (224 pixel) conversion von pixel nach m
# https://confluence.desy.de/display/TPX3BT/2020/12/17/VMI+calibration+with+Simion
tof_offset = 0.926  # arrival time of photon peak

# functions for conversion
r_to_vel = lambda r, tof: (r * c_pixel / vmi_magnification / (tof * 1e-6))  # for m/s
velocity_to_eV = lambda v: 0.5 * v ** 2 * 14.0067 * u * J2eV
pixel_to_velocity = lambda pixel, tof_center: velocity_to_eV(
    r_to_vel(pixel, tof_center)
)


def convert_velocity(df: pd.DataFrame) -> pd.DataFrame:
    """convert absolute coordinate from VMI to relative coordinate and
    calculate velocities
    """
    df["tof"] -= tof_offset - 0.13
    df["x_rel"] = df["x"] - x_cent
    df["y_rel"] = df["y"] - y_cent
    df["r"] = np.sqrt(df["x_rel"] ** 2 + df["y_rel"] ** 2)
    df["theta"] = np.arctan2(
        df["y_rel"], df["x_rel"]
    )  # alternatively: np.arctan2(df['y'], df['r'])

    df["v_x"] = df["x_rel"] * c_pixel / vmi_magnification / (df["tof"] * 1e-6)  # m/s
    df["v_y"] = df["y_rel"] * c_pixel / vmi_magnification / (df["tof"] * 1e-6)  # m/s
    df["xy_velocity"] = r_to_vel(df["r"], df["tof"])

    # conversion only correct for m/q=14 peak, for other peak tof_center needs to be changed
    # convert velocity to energy; E = 0.5 v^2 m
    df["eV"] = velocity_to_eV(df["xy_velocity"])

    Δt = (df["tof"] - tof_center) * 1e-6  # convert to s
    df["v_z"] = Δt * e * E / m

    return df

#### load data

In [ ]:
# load data from standard clustering
file_dbscan = "out/ion-run_0016_20200903-2202.hdf5"
name = os.path.basename(file_dbscan).rstrip(".hdf5")
data_dbscan = convert_velocity(get_data_pd(file_dbscan)[1])
data_raw = convert_velocity(get_data_pd(file_dbscan)[0])

# load data from LoG
file_log_tobi = "out/ion-run_0016_20200903-2202_LoG-Tobi.npy"
data_log_tobi = convert_velocity(
    pd.DataFrame(np.load(file_log_tobi), columns=("nr", "x", "y", "tof", "tot"))
)
file_log_old = "out/ion-run_0016_20200903-2202_LoG-rawConv.npy"
data_log_old = convert_velocity(
    pd.DataFrame(np.load(file_log_old), columns=("nr", "x", "y", "tof", "tot"))
)

# load data from Peer
file_peer = "out/ion-run_0016_20200903-2202_peer.npy"
data_peer = convert_velocity(
    pd.DataFrame(np.load(file_peer), columns=("nr", "x", "y", "tof", "tot"))
)

####
# whole pulse train
# DBSCAN
df = shift_microbunch_pulses(
    get_data_pd(file_dbscan)[1],
    nr_peaks=runNrs[name]["pulses"],
    dt=1 / runNrs[name]["rep"] * 1e3,
    offset=0.9,
)
data_dbscan_all = convert_velocity(df)

# raw data all
df = shift_microbunch_pulses(
    get_data_pd(file_dbscan)[0],
    nr_peaks=runNrs[name]["pulses"],
    dt=1 / runNrs[name]["rep"] * 1e3,
    offset=0.9,
)
data_raw_all = convert_velocity(df)

# LoG Tobi
df = shift_microbunch_pulses(
    pd.DataFrame(np.load(file_log_tobi), columns=("nr", "x", "y", "tof", "tot")),
    nr_peaks=runNrs[name]["pulses"],
    dt=1 / runNrs[name]["rep"] * 1e3,
    offset=0.9,
)
data_log_tobi_all = convert_velocity(df)

# LoG Old
df = shift_microbunch_pulses(
    pd.DataFrame(np.load(file_log_old), columns=("nr", "x", "y", "tof", "tot")),
    nr_peaks=runNrs[name]["pulses"],
    dt=1 / runNrs[name]["rep"] * 1e3,
    offset=0.9,
)
data_log_old_all = convert_velocity(df)


# load data from Peer
df = shift_microbunch_pulses(
    pd.DataFrame(np.load(file_peer), columns=("nr", "x", "y", "tof", "tot")),
    nr_peaks=runNrs[name]["pulses"],
    dt=1 / runNrs[name]["rep"] * 1e3,
    offset=0.9,
)
data_peer_all = convert_velocity(df)

#### analyse data: TOFs

In [ ]:
def TOF_spectrum(data, title="DBSCAN", bins=15_000, alpha=1):
    x_hist, x_bins = np.histogram(data["tof"], bins=bins)
    tof = hv.Histogram((x_hist, x_bins), label=title).opts(
        xlabel="TOF (µs)", title=title
    )
    return tof.opts(alpha=alpha)

In [ ]:
tof_dbscan = TOF_spectrum(data_dbscan.query("tof < 40"), title="DBSCAN", alpha=0.7)
tof_log_tobi = TOF_spectrum(
    data_log_tobi.query("tof < 40"), title="DBSCAN LoG Tobi", alpha=0.5
)
tof_log_old = TOF_spectrum(
    data_log_old.query("tof < 40"), title="DBSCAN LoG Old", alpha=0.4
)
tof_peer = TOF_spectrum(data_peer.query("tof < 40"), title="Peer", alpha=0.9)

In [ ]:
(tof_dbscan + tof_log_old + tof_log_tobi + tof_peer).cols(1)

In [ ]:
tof_raw_all = TOF_spectrum(data_raw_all.query("tof < 5"), title="raw", bins=2_000)
tof_dbscan_all = TOF_spectrum(
    data_dbscan_all.query("tof < 5"), title="DBSCAN", bins=2_000, alpha=0.7
)
tof_log_tobi_all = TOF_spectrum(
    data_log_tobi_all.query("tof < 5"), title="DBSCAN LoG Tobi", bins=2_000, alpha=0.5
)
tof_log_old_all = TOF_spectrum(
    data_log_old_all.query("tof < 5"), title="DBSCAN LoG Old", bins=2_000, alpha=0.5
)
tof_peer_all = TOF_spectrum(data_peer_all.query("tof < 5"), title="Peer", bins=2_000)

In [ ]:
(tof_raw_all * tof_dbscan_all + tof_log_old_all + tof_log_tobi_all + tof_peer_all).cols(
    1
)

In [ ]:
(tof_peer * tof_dbscan * tof_log_tobi * tof_log_old).opts(
    title="m/q=14, 1st pulse", xlim=(2.31, 2.55), ylim=(0, 10_000)
)

In [ ]:
(tof_peer_all * tof_dbscan_all * tof_log_tobi_all * tof_log_old_all).opts(
    title="m/q=14, whole pulse train", xlim=(2.31, 2.55), ylim=(0, 80_000)
)

In [ ]:
(tof_peer * tof_dbscan * tof_log_tobi * tof_log_old).opts(
    title="parent ion, 1st pulse", xlim=(3.34, 3.4), ylim=(0, 15_000)
)

In [ ]:
(tof_peer_all * tof_dbscan_all * tof_log_tobi_all * tof_log_old_all).opts(
    title="parent ion, 1st pulse", xlim=(3.34, 3.5), ylim=(0, 80_000)
)

In [ ]:
def hist_velo_distrib(df: pd.DataFrame, title: str) -> hv.Histogram:
    bins = np.linspace(-15_000, 15_000, 100)

    x_hist, x_bins = np.histogram(df["v_x"], bins=bins)
    hist_vx = hv.Histogram((x_hist, x_bins), label="v_x").opts(
        clim=(0.1, None), title=title, xlabel="velocity (m/s)"
    )
    x_hist, x_bins = np.histogram(df["v_y"], bins=bins)
    hist_vy = hv.Histogram((x_hist, x_bins), label="v_y").opts(
        clim=(0.1, None),
    )
    x_hist, x_edges = np.histogram(df["v_z"], bins=bins)
    hist_vz = hv.Histogram((x_hist, x_edges), label="v_z").opts(xlabel="v_z (m/s)")
    # hist_vz
    return (hist_vx * hist_vy.opts(alpha=0.5) * hist_vz.opts(alpha=0.3)).opts(
        xlabel="velocity (m/s)",
    )

In [ ]:
hist_velo_distrib(
    data_dbscan.query("tof>2.35 & tof<2.55"),
    title="Velocity distribution DBSCAN, 1st pulse",
)

In [ ]:
hist_velo_distrib(
    data_dbscan_all.query("tof>2.35 & tof<2.55"),
    title="Velocity distribution DBSCAN, whole train",
)

In [ ]:
hist_velo_distrib(
    data_log_old.query("tof>2.35 & tof<2.55"),
    title="Velocity distribution LoG - old, 1st pulse",
)

In [ ]:
hist_velo_distrib(
    data_log_old_all.query("tof>2.35 & tof<2.55"),
    title="Velocity distribution LoG - old, whole train",
)

In [ ]:
hist_velo_distrib(
    data_log_tobi.query("tof>2.35 & tof<2.55"),
    title="Velocity distribution LoG - Tobi, 1st pulse",
)

In [ ]:
hist_velo_distrib(
    data_log_tobi_all.query("tof>2.35 & tof<2.55"),
    title="Velocity distribution LoG - Tobi, whole train",
)

In [ ]:
hist_velo_distrib(
    data_peer.query("tof>2.35 & tof<2.55"),
    title="Velocity distribution Peer, 1st pulse",
)

In [ ]:
hist_velo_distrib(
    data_peer_all.query("tof>2.35 & tof<2.55"),
    title="Velocity distribution Peer, whole train",
)

#### 2D VMI

In [ ]:
from scipy.ndimage import gaussian_filter


def hist2D_vmi(
    df: pd.DataFrame,
    p: dict,
    bins=(range(256), range(256)),
    sigma: int = None,
    weights: str = None,
) -> hv.Image:
    weights_data = df[weights] if weights is not None else None
    xy_hist, x_bins, y_bins = np.histogram2d(
        df[p["x"]], df[p["y"]], bins=bins, weights=weights_data
    )
    if sigma is not None:
        image = gaussian_filter(xy_hist.T[::-1], sigma=sigma)
    else:
        image = xy_hist.T[::-1]
    hist2d = hv.Image(
        image, bounds=(x_bins[0], y_bins[0], x_bins[-1], y_bins[-1])
    ).opts(
        axiswise=True,
        logz=True,
        clim=(0.1, None),
        title=p["title"],
        xlabel=p["xlabel"],
        ylabel=p["ylabel"],
    )

    return hist2d  # _slice_1st + hist2d_slice_all

In [ ]:
Δv_z = 1_000
param1 = {
    "title": f"VMI DBSCAN for {-1*Δv_z}<v_z<{Δv_z} m/s, 1st pulse",
    "x": "x",
    "y": "y",
    "xlabel": "pixel",
    "ylabel": "pixel",
}
df1 = data_dbscan.query(f"tof>2.35 & tof<2.55 & v_z>{-1*Δv_z} & v_z<{Δv_z}")
df2 = data_dbscan_all.query(f"tof>2.35 & tof<2.55 & v_z>{-1*Δv_z} & v_z<{Δv_z}")
param2 = param1.copy()
param2["title"] = f"VMI DBSCAN for {-1*Δv_z}<v_z<{Δv_z} m/s, whole pulse train"
bins = (range(256), range(256))
hist2D_vmi(df1, param1, bins) + hist2D_vmi(df2, param2, bins)

In [ ]:
Δv_y = 1000

df1 = data_dbscan.query(f"tof>2.35 & tof<2.55 & v_y>{-1*Δv_y} & v_y<{Δv_y}")
param1 = {
    "title": f"VMI DBSCAN for {-1*Δv_z}<v_z<{Δv_z} m/s, 1st pulse",
    "x": "v_x",
    "y": "v_z",
    "xlabel": "v_x (m/s)",
    "ylabel": "v_z (m/s)",
}
df2 = data_dbscan_all.query(f"tof>2.35 & tof<2.55 & v_y>{-1*Δv_y} & v_y<{Δv_y}")
param2 = param1.copy()
param2["title"] = f"VMI DBSCAN for {-1*Δv_y}<v_z<{Δv_y} m/s, whole pulse train"
bins = np.linspace(-12_000, 12_000, 200)
hist2D_vmi(df1, param1, bins=bins) + hist2D_vmi(df2, param2, bins=bins)

In [ ]:
Δv_z = 1000

df1 = data_log_tobi.query(f"tof>2.35 & tof<2.55 & v_z>{-1*Δv_z} & v_z<{Δv_z}")
param1 = {
    "title": f"LoG Tobi for {-1*Δv_z}<v_z<{Δv_z} m/s, 1st pulse",
    "x": "x",
    "y": "y",
    "xlabel": "pixel",
    "ylabel": "pixel",
}
df2 = data_log_tobi_all.query(f"tof>2.35 & tof<2.55 & v_z>{-1*Δv_z} & v_z<{Δv_z}")
param2 = param1.copy()
param2["title"] = f"LoG Tobi for {-1*Δv_z}<v_z<{Δv_z} m/s, whole pulse train"
bins = (range(256), range(256))
hist2D_vmi(df1, param1, bins=bins) + hist2D_vmi(df2, param2, bins=bins)

In [ ]:
Δv_y = 1000

df1 = data_log_tobi.query(f"tof>2.35 & tof<2.55 & v_y>{-1*Δv_y} & v_y<{Δv_y}")
param1 = {
    "title": f"LoG Tobi for {-1*Δv_z}<v_z<{Δv_z} m/s, 1st pulse",
    "x": "v_x",
    "y": "v_z",
    "xlabel": "v_x (m/s)",
    "ylabel": "v_z (m/s)",
}
df2 = data_log_tobi_all.query(f"tof>2.35 & tof<2.55 & v_y>{-1*Δv_y} & v_y<{Δv_y}")
param2 = param1.copy()
param2["title"] = f"LoG Tobi for {-1*Δv_y}<v_z<{Δv_y} m/s, whole pulse train"
bins = np.linspace(-12_000, 12_000, 200)
hist2D_vmi(df1, param1, bins=bins) + hist2D_vmi(df2, param2, bins=bins)

In [ ]:
Δv_x = 1000
vx_slice = df2.query(f"v_x>{-1*Δv_x} & v_x<{Δv_x}")
x_hist, x_bins = np.histogram(vx_slice["v_z"], bins=(np.linspace(-12_000, 12_000, 200)))
hv.Histogram((x_hist, x_bins)).opts(xlabel="v_z (m/s)")

In [ ]:
Δv_y = 1000

df1 = data_log_old.query(f"tof>2.35 & tof<2.55 & v_y>{-1*Δv_y} & v_y<{Δv_y}")
param1 = {
    "title": f"LoG Old for {-1*Δv_z}<v_z<{Δv_z} m/s, 1st pulse",
    "x": "v_x",
    "y": "v_z",
    "xlabel": "v_x (m/s)",
    "ylabel": "v_z (m/s)",
}
df2 = data_log_old_all.query(f"tof>2.35 & tof<2.55 & v_y>{-1*Δv_y} & v_y<{Δv_y}")
param2 = param1.copy()
param2["title"] = f"LoG Old for {-1*Δv_y}<v_z<{Δv_y} m/s, whole pulse train"
bins = np.linspace(-12_000, 12_000, 200)
hist2D_vmi(df1, param1, bins=bins) + hist2D_vmi(df2, param2, bins=bins)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

df2 = data_log_old_all.query(f"tof>2.35 & tof<2.55 & v_y>{-1*Δv_y} & v_y<{Δv_y}")

fig = plt.figure()
ax = fig.add_subplot()
plt.hist2d(
    df2["v_x"],
    df2["v_z"],
    bins=np.linspace(-12_000, 12_000, 200),
    norm=mpl.colors.LogNorm(),
    cmap=plt.cm.jet,
)
# square plot
ax.set_aspect("equal", adjustable="box")
plt.show()

In [ ]:
Δv_x = 1000
vx_slice = df2.query(f"v_x>{-1*Δv_x} & v_x<{Δv_x}")
x_hist, x_bins = np.histogram(vx_slice["v_z"], bins=(np.linspace(-12_000, 12_000, 200)))
hv.Histogram((x_hist, x_bins)).opts(xlabel="v_z (m/s)")

In [ ]:
Δv_y = 1000

df1 = data_peer.query(f"tof>2.35 & tof<2.55 & v_y>{-1*Δv_y} & v_y<{Δv_y}")
param1 = {
    "title": f"Peer for {-1*Δv_y}<v_y<{Δv_y} m/s, 1st pulse",
    "x": "v_x",
    "y": "v_z",
    "xlabel": "v_x (m/s)",
    "ylabel": "v_z (m/s)",
}
df2 = data_peer_all.query(f"tof>2.35 & tof<2.55 & v_y>{-1*Δv_y} & v_y<{Δv_y}")
param2 = param1.copy()
param2["title"] = f"Peer for {-1*Δv_y}<v_y<{Δv_y} m/s, whole pulse train"
bins = np.linspace(-12_000, 12_000, 200)
hist2D_vmi(df1, param1, bins=bins) + hist2D_vmi(df2, param2, bins=bins)

In [ ]:
# plot TOF as TOT to see if pixels from previous event are still "blind"
Δv_y = 1000

df1 = data_peer_all.query(f"tof>1 & tof<2.7 & v_y>{-1*Δv_y} & v_y<{Δv_y}")
param1 = {
    "title": f"Peer for {-1*Δv_y}<v_y<{Δv_y} m/s",
    "x": "tot",
    "y": "tof",
    "xlabel": "TOT",
    "ylabel": "TOF",
}
bins = (np.arange(0, 3000, 25), np.linspace(1, 2.7, 200))
hist2D_vmi(df1, param1, bins=bins) + hist2D_vmi(
    df1, param1, bins=(np.arange(0, 3000, 25), np.linspace(2.2, 2.7, 200))
)

In [ ]:
# spatial distribution of peak at 1.8µs
df1 = data_peer_all.query(f"tof>1.6 & tof<2 & v_y>{-1*Δv_y} & v_y<{Δv_y}")
df2 = data_peer_all.query(f"tof>2.3 & tof<2.5 & v_y>{-1*Δv_y} & v_y<{Δv_y}")
param1 = {
    "title": f"Spatial distribution proton peak for {-1*Δv_y}<v_y<{Δv_y} m/s",
    "x": "x",
    "y": "y",
    "xlabel": "x (pixel)",
    "ylabel": "y (pixel)",
}
param2 = param1.copy()
param2["title"] = f"Spatial distribution N₂ peak for {-1*Δv_y}<v_y<{Δv_y} m/s"
bins = range(256)
hist2D_vmi(df1, param1, bins=bins) + hist2D_vmi(df2, param2, bins=bins)

In [ ]:
# plot TOF as TOT to see if pixels from previous event are still "blind"
Δv_y = 1000

df1 = data_raw_all.query(f"tof>1 & tof<2.7 & v_y>{-1*Δv_y} & v_y<{Δv_y}")

param1 = {
    "title": f"Raw for {-1*Δv_y}<v_y<{Δv_y} m/s",
    "x": "tot",
    "y": "tof",
    "xlabel": "TOT",
    "ylabel": "TOF",
}
df2 = data_peer_all.query(f"tof>2.2 & tof<2.7 & v_y>{-1*Δv_y} & v_y<{Δv_y}")
param2 = param1.copy()
param2["title"] = f"Peer for {-1*Δv_y}<v_y<{Δv_y} m/s"
bins = (np.arange(0, 3000, 25), np.linspace(2.2, 2.7, 200))
hist2D_vmi(df1, param1, bins=bins) + hist2D_vmi(df2, param2, bins=bins)

In [ ]:
# spatial distribution of peak at 1.8µs
df1 = data_raw_all.query(f"tof>1.6 & tof<2 & v_y>{-1*Δv_y} & v_y<{Δv_y}")
df2 = data_raw_all.query(f"tof>2.3 & tof<2.5 & v_y>{-1*Δv_y} & v_y<{Δv_y}")
param1 = {
    "title": f"Raw spatial distribution proton peak for {-1*Δv_y}<v_y<{Δv_y} m/s",
    "x": "x",
    "y": "y",
    "xlabel": "x (pixel)",
    "ylabel": "y (pixel)",
}
param2 = param1.copy()
param2["title"] = f"Raw Spatial distribution N₂ peak for {-1*Δv_y}<v_y<{Δv_y} m/s"
bins = range(256)
hist2D_vmi(df1, param1, bins=bins) + hist2D_vmi(df2, param2, bins=bins)

In [ ]:
Δv_x = 1000
vx_slice = df2.query(f"v_x>{-1*Δv_x} & v_x<{Δv_x}")
x_hist, x_bins = np.histogram(vx_slice["v_z"], bins=(np.linspace(-12_000, 12_000, 200)))
hv.Histogram((x_hist, x_bins)).opts(xlabel="v_z (m/s)")

In [ ]:
slope1 = (2.49 - 2.38) / 150 * 1e3
slope2 = (2.47 - 2.35) / 150 * 1e3
print(f"slope {slope1:.3f} ns/pixel, {slope1*256:.1f} ns over sensor")
print(f"slope {slope2:.3f} ns/pixel, {slope2*256:.1f} ns over sensor")

In [ ]:
Δv_y = 1000

df2 = data_peer_all.query(f"tof>2.35 & tof<2.55")
p2 = {
    "title": f"Peer whole pulse train",
    "x": "x",
    "y": "tof",
    "xlabel": "x (pixel)",
    "ylabel": "TOF (µs)",
}
bins = (range(256), np.linspace(2.34, 2.51, 256))

# whole pulse train
xy_hist, x_bins, y_bins = np.histogram2d(
    df2["x"],
    df2["tof"],
    bins=bins,
)

hist2d_slice_all = hist2D_vmi(df2, p2, bins=bins)
a = hv.Curve(((100, 200), (2.42, 2.49)))
b = hv.Curve(((100, 172), (2.352, 2.36)))

slope3 = ((2.381 - 2.374) / (172 - 100)) * 1e3
slope4 = ((2.361 - 2.355) / (172 - 100)) * 1e3
c = hv.Curve(((100, 172), (2.374, 2.381)), label=f"{slope3*256:.1f} ns over sensor")
d = hv.Curve(((100, 172), (2.355, 2.361)), label=f"{slope4*256:.1f} ns over sensor")
hv.Layout((hist2d_slice_all * a * b) + hist2D_vmi(df2, p2, bins=bins, sigma=1) * c * d)

In [ ]:
print(f"slope {slope3:.3f} ns/pixel, {slope3*256:.1f} ns over sensor")
print(f"slope {slope4:.3f} ns/pixel, {slope4*256:.1f} ns over sensor")

In [ ]:
Z = xy_hist.T[::-1]

In [ ]:
smoothed = gaussian_filter(Z, sigma=0.7)
bounds = (x_bins[0], y_bins[0], x_bins[-1], y_bins[-1])
hv.Image(smoothed, bounds=bounds).opts(title="Gauss filter (2D)")

In [ ]:
x_axe = get_x_axis_from_bins(y_bins)
a = [hv.Curve((x_axe, Z[:, i])) for i in range(130, 140)]
b = [hv.Curve((x_axe, smoothed[:, i])) for i in range(130, 140)]
hv.Layout(
    hv.Overlay(a).opts(title="raw data", xlabel="TOF (µs)")
    + hv.Overlay(b).opts(title="Gauss smoothed data", xlabel="TOF (µs)")
).cols(1)

In [ ]:
from scipy.signal import savgol_filter

In [ ]:
hv.Curve((x_axe, savgol_filter(Z[:, 135], 9, 3))).opts(
    title="Savgol smoothed"
) * hv.Curve((x_axe, xy_hist.T[::-1][:, 135]))

In [ ]:
smoothed_savgol = [
    hv.Curve((x_axe, savgol_filter(Z[:, i], 9, 3))) for i in range(130, 140)
]
hv.Overlay(smoothed_savgol)

In [ ]:
new_image = [savgol_filter(Z[:, i], 9, 3) for i in range(255)]

In [ ]:
hv.Image(
    np.asarray(new_image).T, bounds=(x_bins[0], y_bins[0], x_bins[-1], y_bins[-1])
).opts(frame_width=400, frame_height=400, title="savgol_filter (1D)")

In [ ]:
from scipy.interpolate import UnivariateSpline

Z_normed = Z / Z.max()
Z_normed.shape

In [ ]:
x = np.linspace(-100, 100, 255)  # x_axe
splined = [UnivariateSpline(x, Z_normed[:, i], s=0.2) for i in range(0, 255)]
a = [hv.Curve((x_axe, i(x))) for i in splined[130:140]]
hv.Overlay(a).opts(xlabel="TOF (µs)")

In [ ]:
curves_2nd_grad = [i.derivative(n=2)(x) for i in splined]
a = [hv.Curve((x_axe, i)) for i in curves_2nd_grad[130:140]]
hv.Overlay(a)

In [ ]:
image_2nd_grad = np.column_stack(curves_2nd_grad)
hv.Image(image_2nd_grad, bounds=bounds).opts(xlim=(120, 150), ylim=(2.35, 2.5))

### Nitrogen, raw data

In [ ]:
TOF_spectrum(data_raw.query("tof < 40"), title="raw")

In [ ]:
Δv_y = 1000

p1 = {
    "title": f"Raw whole pulse train",
    "x": "x",
    "y": "tof",
    "xlabel": "x (pixel)",
    "ylabel": "TOF (µs)",
}
p2 = p1.copy()
p2["x"] = "y"
p2["xlabel"] = "y (pixel)"

df2 = data_raw_all.query(f"tof>2.33 & tof<2.55")
bins = (range(256), np.linspace(2.33, 2.55, 500))

hist2d_slice_all = hist2D_vmi(df2, p2, bins=bins)

x, y = [100, 172], [2.368, 2.374]
slope3 = ((y[1] - y[0]) / (x[1] - x[0])) * 1e3
c = hv.Curve((x, y), label=f"{slope3*256:.1f} ns over sensor")
x, y = [100, 172], [2.346, 2.352]
slope4 = ((y[1] - y[0]) / (x[1] - x[0])) * 1e3
d = hv.Curve((x, y), label=f"{slope4*256:.1f} ns / sensor")
hv.Layout(
    hist2D_vmi(df2, p1, bins=bins, sigma=1) * c * d
    + hist2D_vmi(df2, p2, bins=bins, sigma=1)
)

In [ ]:
df1 = data_raw_all.query(f"tof>2.35 & tof<2.55 & v_y>{-1*Δv_y} & v_y<{Δv_y}")
p1 = {
    "title": f"Raw for {-1*Δv_y}<v_y<{Δv_y} m/s, whole pulse train",
    "x": "x",
    "y": "tof",
    "xlabel": "x (pixel)",
    "ylabel": "TOF (µs)",
}
Δv_x = 1000
df2 = data_raw_all.query(f"tof>2.35 & tof<2.55 & v_x>{-1*Δv_x} & v_x<{Δv_x}")
p2 = {
    "title": f"Raw for {-1*Δv_x}<v_x<{Δv_x} m/s, whole pulse train",
    "x": "y",
    "y": "tof",
    "xlabel": "y (pixel)",
    "ylabel": "TOF (µs)",
}
df3 = data_raw_all.query(f"tof>2.35 & tof<2.55")
p3 = {
    "title": f"Raw",
    "x": "tot",
    "y": "tof",
    "xlabel": "TOT (ns)",
    "ylabel": "TOF (µs)",
}
df4 = data_peer_all.query(f"tof>2.35 & tof<2.55")
p4 = {
    "title": f"Peer",
    "x": "tot",
    "y": "tof",
    "xlabel": "TOT (ns)",
    "ylabel": "TOF (µs)",
}

bins1 = (range(256), np.linspace(2.35, 2.55, 256))
bins2 = (np.arange(0, 3000, 25), np.linspace(2.35, 2.55, 256))

hv.Layout(
    (hist2D_vmi(df1, p1, bins=bins1) + hist2D_vmi(df2, p2, bins=bins1))
    + (hist2D_vmi(df3, p3, bins=bins2) + hist2D_vmi(df4, p4, bins=bins2))
).cols(2)

In [ ]:
df = data_raw.query(f"tof>2.33 & tof<2.55 & x > 100 & x < 170 & y > 100 & y < 170")

In [ ]:
trigger_nr, nr_pixels = np.unique(df["nr"], return_counts=True)
trigger_nr[nr_pixels.argmax()]

In [ ]:
hv.extension("plotly")

In [ ]:
ddf = df.query("nr==55752")
print(ddf.shape)
hv.Scatter3D(ddf, kdims=["x", "y", "tof"], vdims="tot").opts(color="tot", size=5)

In [ ]:
hv.extension("plotly")

In [ ]:
x = np.array((len(x_bins) - 1) * [x_bins[:-1]])
y = np.array((len(y_bins) - 1) * [y_bins[:-1]])
heights = xy_hist.T[::-1]

hv.Surface(heights, bounds=(x_bins[0], y_bins[0], x_bins[-1], y_bins[-1])).opts(
    width=800, height=800, cmap="jet"
)

In [ ]:
x.shape

# proton peak

In [ ]:
hv.extension("bokeh")

In [ ]:
df = data_peer_all.query("tof > 0.5 & tof < 1")
x_hist, x_bins = np.histogram(df["tof"], bins=500)
hv.Histogram((x_hist, x_bins))

In [ ]:
p1 = {
    "title": f"Protons, whole pulse train",
    "x": "x",
    "y": "tof",
    "xlabel": "x (pixel)",
    "ylabel": "TOF (µs)",
}
p2 = p1.copy()
p2["x"] = "y"
p2["xlabel"] = "y (pixel)"
df2 = data_peer_all.query(f"tof > 0.7 & tof < 0.8")
bins = (range(256), np.linspace(0.7, 0.8, 256))

# whole pulse train
xy_hist, x_bins, y_bins = np.histogram2d(
    df2["x"],
    df2["tof"],
    bins=bins,
)
x, y = [100, 179], [0.721, 0.725]
a = hv.Curve((x, y), label=f"{(y[1]-y[0])/(x[1]-x[0])*1e3*256:.1f} ns/256 pixel")
x, y = [100, 179], [0.721, 0.7285]
c = hv.Curve((x, y), label=f"{(y[1]-y[0])/(x[1]-x[0])*1e3*256:.1f} ns/256 pixel")

x, y = [82, 166], [0.727, 0.723]
b = hv.Curve((x, y), label=f"{(y[1]-y[0])/(x[1]-x[0])*1e3*256:.1f} ns/256 pixel")
hist2D_vmi(df2, p1, bins=bins, sigma=0.9) * a * c + hist2D_vmi(
    df2, p2, bins=bins, sigma=0.9
) * b

In [ ]:
p1 = {
    "title": f"Protons raw, whole pulse train",
    "x": "x",
    "y": "tof",
    "xlabel": "x (pixel)",
    "ylabel": "TOF (µs)",
}
p2 = p1.copy()
p2["x"] = "y"
p2["xlabel"] = "y (pixel)"
df2 = data_raw_all.query(f"tof > 0.7 & tof < 0.8")
bins = (range(256), np.linspace(0.7, 0.8, 256))

# whole pulse train
xy_hist, x_bins, y_bins = np.histogram2d(
    df2["x"],
    df2["tof"],
    bins=bins,
)
x, y = [97, 180], [0.716, 0.7185]
a = hv.Curve((x, y), label=f"{(y[1]-y[0])/(x[1]-x[0])*1e3*256:.1f} ns/256 pixel")
x, y = [96, 189], [0.708, 0.71]
c = hv.Curve((x, y), label=f"{(y[1]-y[0])/(x[1]-x[0])*1e3*256:.1f} ns/256 pixel")

x, y = [82, 180], [0.721, 0.718]
b = hv.Curve((x, y), label=f"{(y[1]-y[0])/(x[1]-x[0])*1e3*256:.1f} ns/256 pixel")
hist2D_vmi(df2, p1, bins=bins, sigma=0.9) * a * c + hist2D_vmi(
    df2, p2, bins=bins, sigma=0.9
) * b

# Electrons from Dec 2019

In [ ]:
data_raw, data_cent = get_data_pd("../FlashDec19/out/run_0685_20191217-1533.hdf5")

In [ ]:
df = data_cent.query(f"tof > 0.95 & tof < 0.97")
x_hist, x_bins = np.histogram(df["tof"], bins=100)
x_axe = get_x_axis_from_bins(x_bins)
popt, cov = curve_fit(gauss_fwhm, x_axe, x_hist, p0=[1000, 1, 1])
b = hv.Curve(
    (x_axe, gauss_fwhm(x_axe, *popt)),
    label=f"FWHM {popt[2]*1e3:.1f} ns, x0={popt[1]:.3f} µs",
)

hv.Histogram((x_hist, x_bins)).opts(xlabel="TOF (µs)") * b.opts(
    line_width=2, color="red"
)

In [ ]:
p1 = {
    "title": f"Electrons Dec 2019",
    "x": "x",
    "y": "tof",
    "xlabel": "x (pixel)",
    "ylabel": "TOF (µs)",
}
p2 = p1.copy()
p2["x"] = "y"
p2["xlabel"] = "y (pixel)"
df2 = data_cent.query(f"tof > 0.94 & tof < 0.97")
bins = (range(256), np.linspace(0.955, 0.966, 256))

# whole pulse train
xy_hist, x_bins, y_bins = np.histogram2d(
    df2["x"],
    df2["tof"],
    bins=bins,
)

x, y = [87, 220], [0.9565, 0.9579]
a = hv.Curve((x, y), label=f"{(y[1]-y[0])/(x[1]-x[0])*1e3*256:.1f} ns/256 pixel").opts(
    line_width=1.3, color="red"
)

hist2D_vmi(df2, p1, bins=bins, sigma=1.1) * a + hist2D_vmi(
    df2, p2, bins=bins, sigma=0.9
)

In [ ]:
p1 = {
    "title": f"Electrons, raw Dec 2019",
    "x": "x",
    "y": "tof",
    "xlabel": "x (pixel)",
    "ylabel": "TOF (µs)",
}
p2 = p1.copy()
p2["x"] = "y"
p2["xlabel"] = "y (pixel)"
df2 = data_raw.query(f"tof > 0.8 & tof < 1.2")
bins = (range(256), np.linspace(0.95, 0.985, 256))

x, y = [87, 220], [0.9565, 0.9579]
a = hv.Curve((x, y), label=f"{(y[1]-y[0])/(x[1]-x[0])*1e3*256:.1f} ns/256 pixel").opts(
    line_width=1.3, color="red"
)
hist2D_vmi(df2, p1, bins=bins, sigma=1.1) + hist2D_vmi(df2, p2, bins=bins, sigma=0.9)